<h1 style="border:3px solid black; padding:3%; font-size:314% ; text-align:center; background-color:#0000FF;"> <strong>Classement</strong>

<div style="background-color: #f2f2f2; padding: 24px;">
<p>Le DataFrame appelé <strong>"df_prime"</strong> contient uniquement les candidats issus de la série <strong>STI2D</strong>(on l'as obtenu en chargant un fichier excel contenant que les STI2D mais dont on a modifié les noms de certaines colonnes matières car on arrivait pas à faire la <strong>boucle</strong> dans la fonction de classement avec les floattants <strong>1.1, 2.1 etc..</strong> présents dans le fichier excel d'origine <strong>"PS_Info.xlsl"</strong>), conformément à la décision prise précédemment de  classer que les étudiants de cette filière. En effet, nous ne disposons pas suffisamment de notes pour pouvoir classer les candidats des filières du <strong>baccalauréat général</strong>. Donc nous avons décider de ne classer que les étudiants issues de la filières   STI2D.</p>
    <p> De ce fait nous avons fait 3 classement :
    <ul> <li>un Classement avec tous les STI2D classés indifféremment de l'année d'obtention du bac</li>
        <li>un Classement contenant juste les STI2D nouveaux classés </li>
        <li> un Classement contenant seulement les STI2D anciens</li></ul>
     </p>
 <p>Ainsi on s'est rendu compte que le classement avec juste les STI2D anciens ne contenait qu'un seul candidat. Donc y a juste un bachelier STI2D antérieur qui a réussi à figurer dans le classement. </p>
</div>

In [2]:
import pandas as pd
df_prime= pd.read_excel('st.xlsx')
#df_prime

In [3]:
col_valide_prime= df_prime.dropna(axis=1,how='all')  #supprimme tte les colonnes vident ds le dataframe et met le nouveau dataframe ds la variable col_valide
col_valide_prime= col_valide_prime[col_valide_prime["Candidature validée (O/N)"]!= "Non" ] #ici on recupère que les candidats ayant validé leur candidatures
col_valide_prime=col_valide_prime.reset_index(drop=True)
#col_valide_prime

In [4]:
STI2D_prime= col_valide_prime[col_valide_prime['Série diplôme'].isin(["Sciences et Technologies de l'Industrie et du Développement Durable"])]
STI2D_prime['Série diplôme'].value_counts().sum()

998

In [5]:
STI2D_nouv = STI2D_prime[(STI2D_prime["Niveau d'étude.5"]=="Terminale") & (STI2D_prime["Année scolaire"]=="2021-2022")]
STI2D_nouv['Série diplôme'].value_counts().sum()

814

In [6]:
STI2D_old = STI2D_prime[(STI2D_prime["Niveau d'étude.5"]=="Terminale") & ((STI2D_prime["Année scolaire"]=="2020-2021") | (STI2D_prime["Année scolaire"]=="2019-2020") | (STI2D_prime["Année scolaire"]=="2018-2019") | (STI2D_prime["Année scolaire"]=="2017-2018") | (STI2D_prime["Année scolaire"]=="2016-2017"))]
STI2D_old['Série diplôme'].value_counts().sum()

182

<div style="background-color: #f2f2f2; padding: 30px;">
    <p> Nous avons effectué le classement en créant une fonction(nommée <strong>"classer"</strong>). Cette fonction prend en entrée une DataFrame pandas <strong>(serie)</strong> dont <strong>STI2D</strong> dans notre cas, contenant des informations sur les notes des étudiants dans différentes matières et calcule une nouvelle <strong>note "normalisée"</strong>(càd équilibre la note de l'étudiant par rapport à la note de sa classe et ainsi d'avoir une note de l'élève par rapport au niveau de sa classe) pour chaque étudiant dans chaque matière, basée sur la distribution des notes dans la classe. </p>
  <p>  Ensuite comme dit précédemment nous avons procédé à la normalisation de la note de l'étudiant, sachant qu'on dispose de sa moyenne <strong>x</strong>, de la moyenne de la classe <strong>x_classe</strong>, de la moyenne la plus haute de sa classe <strong>x_max</strong> et enfin de la moyenne la plus basse <strong>x_min</strong>. <br>Le calcul pour la normalisation est modéliséde de la manière suivante :  <strong>x_norm = (x - x_classe) / (x_max - x_min)</strong>  </p>
    <p>Elle calcule ensuite une <strong>moyenne pondérée</strong>(càd chaque moyenne est affectée d'un coefficient) des notes normalisées pour obtenir une <strong>note "globale"</strong> pour chaque étudiant, trie les étudiants par leur note globale, et renvoie une nouvelle DataFrame contenant l'<strong>ID</strong>, la <strong>note globale</strong> une <strong>valeur booléenne</strong> indiquant si l'étudiant est considéré comme <strong>"non-classé"</strong> (c'est-à-dire si sa note globale de la matière est inférieure à un certain seuil pour sa matière <strong>(la planche)</strong>, <strong>département</strong>, <strong>libellé</strong> etc....</p>
</div>

In [7]:
import pandas as pd
def classer(serie, matieres, trimestres, nv_ecart_type, nv_moyenne):
    # Définition des coefficients pour chaque matière
    coef_math = [1, 2, 3, 2, 3, 4]
    coef_français = [1, 1, 1, 1, 1, 1]
    coef_anglais = [1, 1, 1, 1, 1, 1]
    # Création d'un dictionnaire de coefficients pour chaque matière
    coefs = {
        'mathématiques': coef_math,
        'français': coef_français,
        'langue vivante a' : coef_anglais
    }
    serie['les_non_classe']= False
    
    coef_globale_matiere = {'mathématiques':13,'français': 9,'langue vivante a': 4}
    # Parcours de toutes les combinaisons matière/trimestre
    for matiere in matieres:
        for trimestre in trimestres:
            # Calcul des notes normalisées pour la matière et le trimestre actuels
            x_norm = (serie[f'Moyenne candidat en {matiere} Trimestre {trimestre}'] - serie[f'Moyenne classe en {matiere} Trimestre {trimestre}']) / (serie[f'Moyenne plus haute en {matiere} Trimestre {trimestre}'] - serie[f'Moyenne plus basse en {matiere} Trimestre {trimestre}'])
            x_min = x_norm.min()
            x_max = x_norm.max()
            serie[f'normalized_note_{matiere}_T{trimestre}'] = 20 / (x_max - x_min) * (x_norm - x_min)

            # Calcul de la moyenne et de l'écart-type pour la matière et le trimestre actuels
            moyenne = serie[f'normalized_note_{matiere}_T{trimestre}'].mean()
            ecart_type = serie[f'normalized_note_{matiere}_T{trimestre}'].std()

            # Calcul des nouvelles notes normalisées pour la matière et le trimestre actuels
            serie[f'nouvelle_note_normalisee_{matiere}_T{trimestre}'] = (serie[f'normalized_note_{matiere}_T{trimestre}'] - moyenne) / ecart_type * nv_ecart_type + nv_moyenne
    #note_globale_avc_coef=0
    # Calcul des notes globales pour chaque élève
    for index, row in serie.iterrows():
        #intialise la note gloabale
        note_globale=0
        
        for matiere in matieres:
            note_globale_matiere= 0
            coef_matiere = coefs[matiere.lower()]
            for j, trim in enumerate(trimestres):
               #affecter la variable "coef_trimestre" les coefitions de la matiere actuelle 
                coef_trimestre = coef_matiere[j]
                note = serie.loc[index, f'nouvelle_note_normalisee_{matiere}_T{trim}']
                #note_global egale a la note normalisé de la matiere multiplier par le coef de trimestre
                note_globale_matiere += (coef_trimestre * note)
            #note_globale+=note_globale_matiere
            #ici on vas placer les planches si l'eleves va etre en desous de la planche, l'eleve vas etre non classer 
            #C-a-d : 'les_note_classe'=True 
            if(matiere=="Mathématiques"):
                
                note_globale_matiere = note_globale_matiere/sum(coefs['mathématiques'])
                serie.at[index,'note_mAth']=note_globale_matiere
                if (note_globale_matiere <10  or pd.isna(note_globale_matiere)):
                    serie.at[index, 'les_non_classe'] = True
                #la note globale avec coefficion de la matiere    
                note_globale_avc_coef= note_globale_matiere *  coef_globale_matiere['mathématiques']    
            if(matiere=="Français"):
                note_globale_matiere = note_globale_matiere/sum(coefs['français'])
                #cette variable est faite pour calculer la coreelation 
                serie.at[index,'note_fr']=note_globale_matiere
                if (note_globale_matiere < 8 or pd.isna(note_globale_matiere)):
                    serie.at[index, 'les_non_classe'] = True
                note_globale_avc_coef=note_globale_matiere * coef_globale_matiere['français']   
            if(matiere=="Langue vivante A"):
                note_globale_matiere = note_globale_matiere/sum(coefs['langue vivante a'])
                #cette variable est faite pour calculer la coreelation 
                serie.at[index,'note_anglais']=note_globale_matiere
                if (note_globale_matiere <8 or pd.isna(note_globale_matiere)):
                    serie.at[index, 'les_non_classe'] = True
                note_globale_avc_coef=note_globale_matiere*coef_globale_matiere['langue vivante a'] 
                
            #quand on calcule la note globale avec coef  on va l'affecter a une autre variable "note_global" pour cumuler tout les moyenne de l'eleve
            note_globale+= note_globale_avc_coef        
            
            
        #la moyenne sera la note globale qu'est ete cumule avec divisé par la somme des coefition de matieres
        serie.at[index, 'note_globale'] = note_globale/sum(coef_globale_matiere.values())

        
    serie = serie.sort_values('note_globale', ascending=False)
    serie = serie.reset_index(drop=True)
    serie.index += 1

    return serie[['ID','Sexe', 'Boursier', 'Département établissement','Libellé établissement', 'EDS BAC Terminale', 'note_globale','note_mAth','note_fr','note_anglais','les_non_classe', "Niveau d'étude.5","Année scolaire","Série diplôme" ]]

<div id="class"></div>

<h1>Classement indiféremment de l'année d'obtention du bac</h1>


<p>
  <br>
     <br>
    <details open>
        <summary><strong style="font-size:1.414em;color:green;">Menu de navigation<span style="font-size:0.57em;">(cliquez)</span></strong></summary>
        <ul>
            <li><a href="#class">Classement générale (STI2D nouveau et ancien)</a></li>
            <li><a href="#class_nouv">Classement STI2D nouveaux</a></li>
            <li><a href="#class_old">Classement STI2D anciens</a></li>
          </ul>

In [10]:
pd.options.display.max_rows = 998
cla=pd.DataFrame(classer(STI2D_prime, ['Mathématiques', 'Français', 'Langue vivante A'], [1,2,3,4,5,6] , 4, 10))
classers=cla[cla['les_non_classe']==False]

Non_classer=cla[cla['les_non_classe']==True]
Non_classer
classers.head(30) #ça affiche à peu près les 30 premiers du classement

,ID,Sexe,Boursier,Département établissement,Libellé établissement,EDS BAC Terminale,note_globale,note_mAth,note_fr,note_anglais,les_non_classe,Niveau d'étude.5,Année scolaire,Série diplôme
1,1057,M,Boursier du secondaire,93.0,Lycée Eugene Henaff,Physique-Chimie et Mathématiques,16.595764,19.487131,13.543908,14.065500,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
2,2404,M,Boursier du secondaire,95.0,Lycée Simone De Beauvoir,Physique-Chimie et Mathématiques,16.393798,17.494136,14.676686,16.681206,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
3,3154,M,Non boursier,93.0,Lycée CONDORCET,Physique-Chimie et Mathématiques,16.298222,17.879377,14.584378,15.015620,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
4,912,M,Boursier du secondaire,94.0,Lycée Louis Armand,Physique-Chimie et Mathématiques,16.290142,17.178015,15.736320,14.650658,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
5,2918,M,Non boursier,91.0,Lycée Parc de Vilgenis,"Ingénierie, innovation et développement durable",15.697804,18.255966,13.542198,12.233893,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
6,4507,M,Non boursier,95.0,Lycée Jean Perrin,Physique-Chimie et Mathématiques,15.057249,15.785712,13.368262,16.489966,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
7,1356,M,Boursier du secondaire,93.0,Lycée Paul Eluard,Physique-Chimie et Mathématiques,15.053770,16.841523,12.606894,14.749045,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
8,2805,M,Non boursier,60.0,Lycée Jean Monnet,Physique-Chimie et Mathématiques,14.849304,15.608308,13.519511,15.374575,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
9,4253,M,Non boursier,93.0,Lycée Paul Eluard,Physique-Chimie et Mathématiques,14.665877,15.323912,15.609669,10.403731,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
10,4023,F,Non boursier,78.0,Lycée Léonard de Vinci,Physique-Chimie et Mathématiques,14.665150,17.286229,13.724687,8.262688,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...


<div id="class_nouv"></div>

<h1>Classement STI2D nouveaux</h1>


<p>
  <br>
     <br>
    <details open>
        <summary><strong style="font-size:1.414em;color:green;">Menu de navigation<span style="font-size:0.57em;">(cliquez)</span></strong></summary>
        <ul>
            <li><a href="#class">Classement générale (STI2D nouveau et ancien)</a></li>
            <li><a href="#class_nouv">Classement STI2D nouveaux</a></li>
            <li><a href="#class_old">Classement STI2D anciens</a></li>
          </ul>

In [13]:

cla=pd.DataFrame(classer(STI2D_nouv, ['Mathématiques', 'Français', 'Langue vivante A'], [1,2,3,4,5,6] , 4, 10))
classers_STI2D_nouv=cla[cla['les_non_classe']==False]

Non_classer=cla[cla['les_non_classe']==True]
Non_classer
classers_STI2D_nouv.head(30)

/tmp/ipykernel_19102/69634845.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['les_non_classe']= False
/tmp/ipykernel_19102/69634845.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie[f'normalized_note_{matiere}_T{trimestre}'] = 20 / (x_max - x_min) * (x_norm - x_min)
/tmp/ipykernel_19102/69634845.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

,ID,Sexe,Boursier,Département établissement,Libellé établissement,EDS BAC Terminale,note_globale,note_mAth,note_fr,note_anglais,les_non_classe,Niveau d'étude.5,Année scolaire,Série diplôme
1,1057,M,Boursier du secondaire,93.0,Lycée Eugene Henaff,Physique-Chimie et Mathématiques,16.510448,19.342351,13.506154,14.066422,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
2,2404,M,Boursier du secondaire,95.0,Lycée Simone De Beauvoir,Physique-Chimie et Mathématiques,16.326895,17.394710,14.621093,16.694547,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
3,912,M,Boursier du secondaire,94.0,Lycée Louis Armand,Physique-Chimie et Mathématiques,16.224785,17.088533,15.673800,14.657324,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
4,3154,M,Non boursier,93.0,Lycée CONDORCET,Physique-Chimie et Mathématiques,16.223585,17.755543,14.546249,15.018730,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
5,2918,M,Non boursier,91.0,Lycée Parc de Vilgenis,"Ingénierie, innovation et développement durable",15.634671,18.148001,13.515432,12.234637,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
6,1356,M,Boursier du secondaire,93.0,Lycée Paul Eluard,Physique-Chimie et Mathématiques,15.010012,16.758164,12.594837,14.762661,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
7,4507,M,Non boursier,95.0,Lycée Jean Perrin,Physique-Chimie et Mathématiques,14.998372,15.696633,13.326254,16.491288,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
8,2805,M,Non boursier,60.0,Lycée Jean Monnet,Physique-Chimie et Mathématiques,14.800692,15.535015,13.481433,15.382476,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
9,4253,M,Non boursier,93.0,Lycée Paul Eluard,Physique-Chimie et Mathématiques,14.606162,15.248806,15.546442,10.401940,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...
10,4023,F,Non boursier,78.0,Lycée Léonard de Vinci,Physique-Chimie et Mathématiques,14.601418,17.186979,13.690343,8.248267,False,Terminale,2021-2022,Sciences et Technologies de l'Industrie et du ...


<div id="class_old"></div>

<h1>Classement STI2D anciens</h1>


<p>
  <br>
     <br>
    <details open>
        <summary><strong style="font-size:1.414em;color:green;">Menu de navigation<span style="font-size:0.57em;">(cliquez)</span></strong></summary>
        <ul>
            <li><a href="#class">Classement générale (STI2D nouveau et ancien)</a></li>
            <li><a href="#class_nouv">Classement STI2D nouveaux</a></li>
            <li><a href="#class_old">Classement STI2D anciens</a></li>
          </ul>

In [11]:
pd.options.display.max_rows = 998
cla=pd.DataFrame(classer(STI2D_old, ['Mathématiques', 'Français', 'Langue vivante A'], [1,2,3,4,5,6] , 4, 10))

classers_STI2D_old=cla[cla['les_non_classe']==False]

Non_classer=cla[cla['les_non_classe']==True]
Non_classer
classers_STI2D_old

/tmp/ipykernel_4283/69634845.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie['les_non_classe']= False
/tmp/ipykernel_4283/69634845.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serie[f'normalized_note_{matiere}_T{trimestre}'] = 20 / (x_max - x_min) * (x_norm - x_min)
/tmp/ipykernel_4283/69634845.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

,ID,Sexe,Boursier,Département établissement,Libellé établissement,EDS BAC Terminale,note_globale,note_mAth,note_fr,note_anglais,les_non_classe,Niveau d'étude.5,Année scolaire,Série diplôme
1,619,M,Non boursier,13.0,IUT Aix-Marseille - Site d'Aix-En-Provence,NaN,14.71415,17.05694,12.122758,12.930713,False,Terminale,2018-2019,Sciences et Technologies de l'Industrie et du ...
